## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [ ]:
!pip install sqlalchemy==1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
!pip install -q pandas==1.1.5

In [3]:
%sql sqlite:///my_data1.db

In [4]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

C:\Users\ibayl\anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


### Display the names of the unique launch sites  in the space mission


In [5]:
%sql select distinct(Launch_Site) from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [6]:
%%sql SELECT * from SPACEXTBL
WHERE Launch_Site LIKE '%CCA%'
LIMIT 5

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [7]:
%%sql SELECT SUM(PAYLOAD_MASS__KG_) FROM SPACEXTBL
WHERE Customer == 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
45596


In [8]:
%%sql SELECT AVG(PAYLOAD_MASS__KG_) FROM SPACEXTBL
WHERE Booster_Version LIKE '%F9 v1.1%'

 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2534.6666666666665


In [9]:
%%sql SELECT MIN(Date) FROM SPACEXTBL
WHERE Launch_Outcome == 'Success (ground pad)'

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such column: Launch_Outcome
[SQL: SELECT MIN(Date) FROM SPACEXTBL
WHERE Launch_Outcome == 'Success (ground pad)']
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [10]:
%%sql SELECT Booster_Version FROM SPACEXTBL
WHERE (Landing_Outcome == 'Success (drone ship)') AND (PAYLOAD_MASS__KG_<6000) AND (PAYLOAD_MASS__KG_>4000)

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such column: Landing_Outcome
[SQL: SELECT Booster_Version FROM SPACEXTBL
WHERE (Landing_Outcome == 'Success (drone ship)') AND (PAYLOAD_MASS__KG_<6000) AND (PAYLOAD_MASS__KG_>4000)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [11]:
%%sql SELECT Mission_Outcome, count(Mission_Outcome) AS Count FROM SPACEXTBL
GROUP BY Mission_Outcome

 * sqlite:///my_data1.db
Done.


Mission_Outcome,Count
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [12]:
%%sql SELECT Booster_Version FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ IN (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [13]:
%%sql SELECT substr(Date, 4, 2), Landing_Outcome, Booster_Version, Launch_Site FROM SPACEXTBL
WHERE Date IN (SELECT substr(Date,7,4) as YEAR FROM SPACEXTBL WHERE YEAR == 2015) AND (Landing_Outcome == 'Failure (drone ship)')

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such column: Landing_Outcome
[SQL: SELECT substr(Date, 4, 2), Landing_Outcome, Booster_Version, Launch_Site FROM SPACEXTBL
WHERE Date IN (SELECT substr(Date,7,4) as YEAR FROM SPACEXTBL WHERE YEAR == 2015) AND (Landing_Outcome == 'Failure (drone ship)')]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [14]:
%%sql SELECT Landing_Outcome, Count(Landing_Outcome) FROM SPACEXTBL
WHERE Landing_Outcome LIKE '%Success%'
GROUP BY Landing_Outcome
ORDER BY Landing_Outcome

 * sqlite:///my_data1.db
(sqlite3.OperationalError) no such column: Landing_Outcome
[SQL: SELECT Landing_Outcome, Count(Landing_Outcome) FROM SPACEXTBL
WHERE Landing_Outcome LIKE '%Success%'
GROUP BY Landing_Outcome
ORDER BY Landing_Outcome]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
